In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

### Read the sampled dataset with Amazon inventory data

In [3]:
df_items = pd.read_json("../../data/meta_Video_Games_2022_2023_with_category_ratings_10_sample_1000.jsonl", lines=True)

In [4]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,Computers,Mad Catz S.T.R.I.K.E. 13 Compact Premium Mecha...,4.4,52,[96-key rollover - never miss a keystroke with...,[Save space without sacrificing performance wi...,117.21,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Mad Catz S.T.R.I.K.E. 13 Compact W...,Mad Catz,"[Video Games, PC, Accessories, Gaming Keyboards]","{'Brand': 'Mad Catz', 'Series': 'S.T.R.I.K.E. ...",B0BLVNZNWV,NaN
1,All Electronics,PS5 Console Plate with Vent and RGB LED Light ...,4.7,59,[🔴【VENTILATION WINDOWS】SIKEMAY especially unde...,[],59.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'PS5 Plates Installation Video', 'u...",SIKEMAY,"[Video Games, PlayStation 5, Accessories, Cont...",{'Package Dimensions': '15.51 x 10.63 x 2.52 i...,B0BM4L3J9Q,NaN
2,Health & Personal Care,"XP-RX Gamer Supplement for Energy, Focus & End...",4.1,27,"[All the Boost, None of the Crash: Through ext...",[],29.95,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Honest Review of Ghost Gamer for F...,DR EMIL NUTRITION,"[Video Games, Legacy Systems, Nintendo Systems]","{'Brand': 'DR EMIL NUTRITION', 'Unit Count': '...",B0BZBP1LF4,NaN
3,All Electronics,SAMINRA Replacement for Xbox One Wired Control...,3.1,12,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Xbox 360 Wired Controller Unboxing...,SAMINRA,"[Video Games, Xbox One, Accessories, Controlle...",{'Package Dimensions': '6.3 x 5.24 x 2.87 inch...,B0BCJJX6GN,NaN
4,Cell Phones & Accessories,GAMSURFING Silicone Dust Plug Cover for Xbox S...,4.5,37,[ONLY FIT FOR XBOX SERIES X/S: The rubber dust...,[],8.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Dust Proof Cooling Fan for Xbox Se...,GAMSURFING,"[Video Games, Xbox Series X & S, Accessories, ...",{'Package Dimensions': '6.42 x 1.89 x 0.47 inc...,B0B1BYBZRP,NaN


In [5]:
list(df_items["features"].items())[0]

(0,
 ['96-key rollover - never miss a keystroke with full-key anti-ghosting features. Ultra-reliable Cherry MX Red mechanical switches - 50 million strike life',
  "Double-shot injection molding with aluminum faceplate creates a durable keyboard that won't wear down",
  'Compact, space-saving design, ideal for small setups or travelling to tournaments. Integrated 3-way cable management keeps your space clutter-free and included keycap puller tool makes cleaning quick and easy',
  'Included exclusive software lets you customize your settings create macros, customize lighting with up to 18 different effects and more. For Windows only, available for download',
  'Multimedia and lighting shortcut keys provide instant access to popular commands. Independent RGB LED in every key - personalize your gaming experience with up to 16.8 million customizable colors'])

In [6]:
list(df_items["images"].items())[0]

(0,
 [{'thumb': 'https://m.media-amazon.com/images/I/41nxTtCkjyL._AC_US40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/41nxTtCkjyL._AC_.jpg',
   'variant': 'MAIN',
   'hi_res': 'https://m.media-amazon.com/images/I/812MGZa3F9L._AC_SL1500_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/5171PBZkGHL._AC_US40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/5171PBZkGHL._AC_.jpg',
   'variant': 'PT01',
   'hi_res': 'https://m.media-amazon.com/images/I/81uEwx7MQIL._AC_SL1500_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51Sw13-PLzL._AC_US40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51Sw13-PLzL._AC_.jpg',
   'variant': 'PT02',
   'hi_res': 'https://m.media-amazon.com/images/I/81OKsey9F8L._AC_SL1500_.jpg'},
  {'thumb': 'https://m.media-amazon.com/images/I/51MDsH1+dwL._AC_US40_.jpg',
   'large': 'https://m.media-amazon.com/images/I/51MDsH1+dwL._AC_.jpg',
   'variant': 'PT03',
   'hi_res': 'https://m.media-amazon.com/images/I/91-pF9l5l4L._AC_SL1

### Preprocess title and features

In [7]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [8]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [9]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [10]:
df_items.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,image
0,Computers,Mad Catz S.T.R.I.K.E. 13 Compact Premium Mecha...,4.4,52,[96-key rollover - never miss a keystroke with...,Mad Catz S.T.R.I.K.E. 13 Compact Premium Mecha...,117.21,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Mad Catz S.T.R.I.K.E. 13 Compact W...,Mad Catz,"[Video Games, PC, Accessories, Gaming Keyboards]","{'Brand': 'Mad Catz', 'Series': 'S.T.R.I.K.E. ...",B0BLVNZNWV,NaN,https://m.media-amazon.com/images/I/41nxTtCkjy...
1,All Electronics,PS5 Console Plate with Vent and RGB LED Light ...,4.7,59,[🔴【VENTILATION WINDOWS】SIKEMAY especially unde...,PS5 Console Plate with Vent and RGB LED Light ...,59.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'PS5 Plates Installation Video', 'u...",SIKEMAY,"[Video Games, PlayStation 5, Accessories, Cont...",{'Package Dimensions': '15.51 x 10.63 x 2.52 i...,B0BM4L3J9Q,NaN,https://m.media-amazon.com/images/I/51vDfmLEhs...
2,Health & Personal Care,"XP-RX Gamer Supplement for Energy, Focus & End...",4.1,27,"[All the Boost, None of the Crash: Through ext...","XP-RX Gamer Supplement for Energy, Focus & End...",29.95,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Honest Review of Ghost Gamer for F...,DR EMIL NUTRITION,"[Video Games, Legacy Systems, Nintendo Systems]","{'Brand': 'DR EMIL NUTRITION', 'Unit Count': '...",B0BZBP1LF4,NaN,https://m.media-amazon.com/images/I/41zAXnbyHm...
3,All Electronics,SAMINRA Replacement for Xbox One Wired Control...,3.1,12,[],SAMINRA Replacement for Xbox One Wired Control...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Xbox 360 Wired Controller Unboxing...,SAMINRA,"[Video Games, Xbox One, Accessories, Controlle...",{'Package Dimensions': '6.3 x 5.24 x 2.87 inch...,B0BCJJX6GN,NaN,https://m.media-amazon.com/images/I/41MQmHyyhD...
4,Cell Phones & Accessories,GAMSURFING Silicone Dust Plug Cover for Xbox S...,4.5,37,[ONLY FIT FOR XBOX SERIES X/S: The rubber dust...,GAMSURFING Silicone Dust Plug Cover for Xbox S...,8.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'Dust Proof Cooling Fan for Xbox Se...,GAMSURFING,"[Video Games, Xbox Series X & S, Accessories, ...",{'Package Dimensions': '6.42 x 1.89 x 0.47 inc...,B0B1BYBZRP,NaN,https://m.media-amazon.com/images/I/31JF3Yk0GD...


In [11]:
list(df_items["description"].items())[0]

(0,
 "Mad Catz S.T.R.I.K.E. 13 Compact Premium Mechanical Wired Gaming Keyboard with Aluminum Frame Cherry MX RED switches and RGB Lighting 96-key rollover - never miss a keystroke with full-key anti-ghosting features. Ultra-reliable Cherry MX Red mechanical switches - 50 million strike life Double-shot injection molding with aluminum faceplate creates a durable keyboard that won't wear down Compact, space-saving design, ideal for small setups or travelling to tournaments. Integrated 3-way cable management keeps your space clutter-free and included keycap puller tool makes cleaning quick and easy Included exclusive software lets you customize your settings create macros, customize lighting with up to 18 different effects and more. For Windows only, available for download Multimedia and lighting shortcut keys provide instant access to popular commands. Independent RGB LED in every key - personalize your gaming experience with up to 16.8 million customizable colors")

### Sample 50 items from the dataset

In [12]:
df_sample = df_items.sample(50, random_state=42)

In [13]:
len(df_sample)

50

In [14]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [15]:
data_to_embed

[{'description': "Light up Headphone Controller Holder, 16 Colors 3D LED Lights Gamepad Headset Stand, Game Controller Hanger for All Universal Gaming PC Accessories 👍NOT ONLY a STAND: Multi advanced all-round stand: controller holder+headset stand+...---all universal models can be perfectly stored or displayed.Excellent stable and flexible, compatibility with your stylish interiors and digital home appliances , so it not only a stand but also a night light. 👍Creative 3D Visual Effect: This is a 3D illusion lamp that can show up the gamepad when it lights up, with an optical acrylic flat board with laser engraving can give you amazing 3D visual effect. 3D is just visual, the light itself is flat. It's bound to amaze and impress your kids and guests. It is a unique birthday, Christmas, or Easter gift for kids or friends, controller decorations for home. 👍Convenient and Safe: The 3D gamepad night light could use USB to connect or 3 x AAA batteries (not included) to power. The light sourc

### Define the embedding function

In [16]:
response = openai.embeddings.create(
    input="Random text",
    model="text-embedding-3-small",
)

In [17]:
len(response.data[0].embedding)

1536

In [18]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [19]:
get_embedding("Hi")

[-0.006908939220011234,
 -0.035309892147779465,
 0.0016163368709385395,
 0.0653558298945427,
 0.03294406831264496,
 -0.024205315858125687,
 -0.026097973808646202,
 0.049416106194257736,
 0.016220666468143463,
 -0.051781926304101944,
 -0.0134260393679142,
 -0.014520231634378433,
 -0.025994468480348587,
 -0.0032567016314715147,
 0.02461933344602585,
 0.0011505656875669956,
 -0.05349714681506157,
 0.015074721537530422,
 0.011466843076050282,
 0.03400868922472,
 0.049356959760189056,
 0.02034606784582138,
 -0.01405446045100689,
 0.018897002562880516,
 0.01712263561785221,
 0.024234889075160027,
 0.01826118864119053,
 -0.0011718211462721229,
 0.019577177241444588,
 -0.03678853064775467,
 0.027694903314113617,
 -0.0281976405531168,
 0.027620971202850342,
 -0.016250239685177803,
 -0.011799536645412445,
 -0.016013657674193382,
 -0.014120999723672867,
 0.0375869944691658,
 0.01888221688568592,
 -0.03764614090323448,
 0.04344240576028824,
 -0.01241317205131054,
 0.020937524735927582,
 0.01355172

### Create Qdrant collection

In [20]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [21]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

### Embed data

##### Test

In [22]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("Test text"),
    payload={
        "text": "Test text",
        "model": "text-embedding-3-small",
    },
)

In [23]:
pointstruct

PointStruct(id=0, vector=[-0.020057253539562225, 0.006970119196921587, 0.037700485438108444, -0.040323127061128616, -0.01916317082941532, -0.0343029722571373, 0.0005480913096107543, -0.02425944246351719, 0.03871377930045128, 0.0015953787369653583, 0.030667034909129143, 0.012792832218110561, -0.010863103903830051, 0.011287793517112732, 0.02639034017920494, 0.04303517937660217, -0.046820126473903656, -0.00690678833052516, -0.02174110896885395, 0.05108192190527916, 0.0070856050588190556, 0.013113211840391159, 0.01130269467830658, -0.032544609159231186, -0.00393396383151412, -0.039041608572006226, -0.036955416202545166, 0.004965884145349264, 0.05841339752078056, -0.07462609559297562, 0.031382299959659576, -0.044018667191267014, -0.00025891143013723195, -0.010974864475429058, 0.006005255039781332, 0.03814752772450447, 0.03030940145254135, 0.037640880793333054, 0.010274499654769897, -0.029355714097619057, -0.014916278421878815, -0.010386260226368904, 0.021279167383909225, 0.01814987696707248

### Amazon data

In [24]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embediing = get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embediing,
            payload=data,
        )
    )

In [25]:
pointstructs

[PointStruct(id=0, vector=[0.01709727570414543, 0.008701291866600513, -0.015887783840298653, -0.01476049143821001, 0.011049819178879261, -0.07200583815574646, -0.011748505756258965, 0.02114848420023918, 0.03161117061972618, -0.040347691625356674, 0.010732768103480339, -0.010204349644482136, -0.02484741434454918, -0.015512020327150822, 0.025692883878946304, 0.0012094913981854916, -0.057773761451244354, -7.935451867524534e-05, 0.004042401909828186, 0.03365439176559448, 0.03459380194544792, 0.026233045384287834, 0.018952611833810806, 0.026162588968873024, 0.03424152359366417, 0.006693301722407341, 0.039196912199258804, -0.0033760073129087687, 0.008519281633198261, 0.0385158397257328, -0.010803223587572575, -0.014971858821809292, -0.007104293908923864, -0.05824346840381622, -0.03168162703514099, 0.0479334332048893, -0.021430307999253273, -0.010985234752297401, -0.028675513342022896, -0.01398547738790512, 0.020032934844493866, 0.010744510218501091, 0.017884032800793648, 0.000336683355271816

In [26]:
len(pointstructs)

50

### Write embedded data to Qdrant

In [27]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Define a function for data retrieval

In [28]:
def retrieve_data(query, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

### Test retrieval

In [30]:
retrieve_data("What kind of controllers do you offer?", k=10).points

[ScoredPoint(id=36, version=1, score=0.38028875, payload={'description': "iTwano Controller Grips Compatible with Oculus Quest 2, Silicone Controller Cover Protector for Quest 2 with Adjustable Knuckle Strap Compatible with Quest 2: We special design these touch controller grips for quest 2. It will secure controllers to your hands, allowing you to avoid holding them all the gaming time, preventing them from slipping off accidentally, and providing a more immersive experience. Humanized design: The grip covers are made of durable, comfortable, and breathable silicone. The texture of the grips improves grip and air ventilation, resulting in a better grip and less sweaty hands. Excellent Adjustability: The soft leather belt and magic tape will help you find the tightness you need. The rotatable rivet allows you to adjust the angle depending on your palm size. Upgraded Design: We analyzed the shortcomings of the previous product and updated our product. Because this new controller grip do